#### IPTW Modeling

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

# 구성요소 리스트 -- 정의 필요
treatment = 'treatment'
target = 'target'
covs = ['age', 'sex', 'smoke', 'drink', 'exercise']
continuous_covs = ['age']
categorical_covs = ['sex', 'smoke', 'drink', 'exercise']

results = []

df_iptw = data[['age', 'sex', 'smoke', 'drink', 'exercise', treatment, target]].copy()
df_iptw = df_iptw.dropna(subset=[treatment, target] + covs)
df_iptw['treatment_group'] = df_iptw[treatment].astype(float)

# Propensity Score Model
X_cat = pd.get_dummies(df_iptw[categorical_covs], drop_first=True).astype(float)
X = pd.concat([df_iptw[continuous_covs], X_cat], axis=1)
X_const = sm.add_constant(X)

ps_model = sm.Logit(df_iptw['treatment_group'], X_const).fit(disp=0)
df_iptw['pscore'] = ps_model.predict(X_const)

# IPTW 계산
df_iptw['weight'] = df_iptw.apply(
    lambda row: 1 / row['pscore'] if row['treatment_group'] == 1 else 1 / (1 - row['pscore']),
    axis=1
)

# ▶ Trimming (0.01–0.99 범위 유지)
df_iptw_trim = df_iptw[(df_iptw['pscore'] >= 0.01) & (df_iptw['pscore'] <= 0.99)]

treated = df_iptw_trim[df_iptw_trim['treatment_group'] == 1]
control = df_iptw_trim[df_iptw_trim['treatment_group'] == 0]

rate1 = np.average(treated[target], weights=treated['weight'])
rate0 = np.average(control[target], weights=control['weight'])
diff = rate1 - rate0

# Table 요소
n1 = f"{int(np.sum(treated[target])):,} / {len(treated):,} ({rate1*100:.1f}%)"
n0 = f"{int(np.sum(control[target])):,} / {len(control):,} ({rate0*100:.1f}%)"

X_logit = sm.add_constant(df_iptw_trim['treatment_group'])
logit_model = sm.Logit(df_iptw_trim[target], X_logit)
result = logit_model.fit(weights=df_iptw_trim['weight'], disp=0)

coef = result.params['treatment_group']
se = result.bse['treatment_group']
or_val = np.exp(coef)
ci_low = np.exp(coef - 1.96 * se)
ci_high = np.exp(coef + 1.96 * se)
pval = result.pvalues['treatment_group']

results.append({
    'Exposure=0 Cases': n0,
    'Exposure=1 Cases': n1,
    'OR (95% CI)': f'{or_val:.3f} ({ci_low:.3f}–{ci_high:.3f})',
    'p-value': f'{pval:.3f}' if pval >= 0.001 else '<0.001'
})

df_iptw_trimmed_results = pd.DataFrame(results)
df_iptw_trimmed_results

#### KDE plot

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(6, 4))

sns.kdeplot(df_iptw[df_iptw['treatment_group'] == 1]['pscore'], label=f'{treatment.upper()} = 1', fill=True, color='red')
sns.kdeplot(df_iptw[df_iptw['treatment_group'] == 0]['pscore'], label=f'{treatment.upper()} = 0', fill=True, color='blue')

# trimming line 표시
plt.axvline(0.01, color='gray', linestyle='--')
plt.axvline(0.99, color='gray', linestyle='--')
plt.title('Treatment PS Overlap', fontsize=11)
plt.xlabel('Propensity Score', fontsize=10)
plt.ylabel('Density', fontsize=10)
plt.legend(loc='upper right', fontsize=9)

plt.tight_layout()
plt.show()

#### E-value calculate

In [ ]:
def e_value(or_estimate):
    if or_estimate < 1:
        or_estimate = 1 / or_estimate
    return or_estimate + np.sqrt(or_estimate * (or_estimate - 1))

df_iptw_trimmed_results['E-value'] = df_iptw_trimmed_results['OR (95% CI)'].str.extract(r'([\d\.]+)').astype(float).apply(lambda x: round(e_value(x[0]), 3), axis=1)
df_iptw_trimmed_results

#### Forestplot

In [ ]:
import seaborn as sns

plt.figure(figsize=(6, 4))
sns.histplot(df_iptw['weight'], bins=30, kde=True)
plt.title('Distribution of IPTW Weights')
plt.xlabel('Weight')
plt.ylabel('Frequency')
plt.tight_layout()
plt.show()

#### SMD comparison

In [ ]:
# 더미화
df_smd = pd.get_dummies(df_iptw, columns=['sex', 'smoke', 'drink', 'exercise'], drop_first=True)

# 더미화된 컬럼만 필터링
smd_cols = [col for col in df_smd.columns if col.startswith(('age', 'sex_', 'smoke_', 'drink_', 'exercise_'))]

# SMD 계산
before_smd = compute_smd(df_smd, 'treatment_group', smd_cols)

# Trimming 이후도 동일하게 처리
df_smd_trim = pd.get_dummies(df_iptw_trim, columns=['sex', 'smoke', 'drink', 'exercise'], drop_first=True)
smd_cols_trim = [col for col in df_smd_trim.columns if col in smd_cols]  # 동일 컬럼만 유지
after_smd = compute_smd(df_smd_trim, 'treatment_group', smd_cols_trim, weight='weight')

# 시각화
smd_df = pd.DataFrame({'Before IPTW': before_smd, 'After IPTW': after_smd})
smd_df.plot(kind='barh', figsize=(8, 5))
plt.axvline(0.1, color='red', linestyle='--', label='SMD=0.1 threshold')
plt.legend()
plt.title('Covariate Balance: Before vs After IPTW')
plt.xlabel('Standardized Mean Difference (SMD)')
plt.tight_layout()
plt.show()